In [ ]:
# append path to find utils module in urbansim
import os 
import sys
cwd = os.getcwd() 
parentdir =  os.path.abspath(os.path.join(cwd, os.pardir))
sys.path.append(parentdir) # to get path to utils module

In [ ]:
import pandas as pd
import os, datetime
import numpy as np
import utils
import orca
import matplotlib.pyplot as plt
from pandas.plotting import table
import matplotlib.patches as mpatches
from sqlalchemy import create_engine
from database import get_connection_string
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.patches import Rectangle
%matplotlib inline

In [ ]:
db_connection_string = get_connection_string('..\data\config.yml', 'mssql_db')
mssql_engine = create_engine(db_connection_string)
versions = utils.yaml_to_dict('../data/scenario_config.yaml', 'scenario')

## Get subregional simulation output

In [ ]:
# get max run id from urbansim
run_id_sql = '''
SELECT max(run_id)
  FROM [urbansim].[urbansim].[urbansim_lite_output]
'''
run_id_df = pd.read_sql(run_id_sql, mssql_engine)
run_id = int(run_id_df.values)
print("\n   Max run id : {:,}".format(run_id))

hs_change_sql = '''
    SELECT o.parcel_id, j.name,  p.cap_jurisdiction_id, p.jurisdiction_id, p.mgra_id, p.luz_id,
    unit_change as hs_change, source, capacity_type, year_simulation
      FROM urbansim.urbansim.urbansim_lite_output o 
      JOIN urbansim.urbansim.parcel p on p.parcel_id = o.parcel_id
      JOIN urbansim.ref.jurisdiction j on p.cap_jurisdiction_id = j.jurisdiction_id
     WHERE run_id =  %s
  ORDER BY j.name,p.jurisdiction_id, year_simulation'''
hs_change_sql = hs_change_sql % run_id
hs = pd.read_sql(hs_change_sql,mssql_engine)
print("\n   Units added: {:,}".format(int(hs.hs_change.sum())))

##### replace sgoa subtypes with 'sgoa'

In [ ]:
hs.replace('cc', 'sgoa',inplace=True)
hs.replace('mc', 'sgoa',inplace=True)
hs.replace('tc', 'sgoa',inplace=True)
hs.replace('tco', 'sgoa',inplace=True)
hs.replace('uc', 'sgoa',inplace=True)

### Summary

In [ ]:

cap_df = pd.DataFrame({'units_change': hs.groupby(['cap_jurisdiction_id','capacity_type','name']).\
                       hs_change.sum()}).reset_index()

cap_summary = cap_df.pivot_table(index=['cap_jurisdiction_id','name'], columns=['capacity_type'], \
                         values=['units_change'])
cap_summary.fillna(0,inplace=True)


In [ ]:
cap_summary.columns = cap_summary.columns.droplevel(0)
cap_summary.columns.name = None

In [ ]:
cols = ['sch','jur','adu','sgoa']
cap_summary[cols] = cap_summary[cols].applymap(np.int64)
cap_summary['total units'] = cap_summary['sch']  + cap_summary['jur'] + cap_summary['adu'] + cap_summary['sgoa']
cap_summary.rename(columns={"sch": "schdev", "jur": "jur feedback"},inplace=True)

In [ ]:
cap_summary.reset_index(inplace=True)
# cap_summary.head()

In [ ]:
cap_summary['jur_provided_total']= cap_summary['schdev'] + cap_summary['jur feedback']
cap_summary['additional_capacity_total']= cap_summary['sgoa'] + cap_summary['adu']
cap_summary.loc['Total']= cap_summary.sum()
cap_summary.loc[cap_summary.index=='Total','name'] = 'Region'

In [ ]:
cap_summary.head()

In [ ]:
cap_summary.set_index('cap_jurisdiction_id',inplace=True)

In [ ]:
cap_summary[['name','schdev','jur feedback','adu','sgoa','total units','jur_provided_total','additional_capacity_total']]

### QC check output against jurisdiction feedback confluence pg (since all jur capacity used)

https://sandag.atlassian.net/wiki/spaces/LUM/pages/101679105/Jurisdictional+Feedback

## Get total dwelling units in the region and sum by jurisdiction and cpa

#### note using cap jurisdiction id

In [ ]:
du_sql = '''
    SELECT parcel_id, mgra_id, luz_id, p.jurisdiction_id, cap_jurisdiction_id, j.name, du_2015 as du, du_2017, capacity_2
        FROM urbansim.parcel p
        LEFT JOIN urbansim.ref.jurisdiction j on p.cap_jurisdiction_id = j.jurisdiction_id'''
#        WHERE du_2017 > 0'''
du = pd.read_sql(du_sql,mssql_engine)
du.cap_jurisdiction_id.fillna(du.jurisdiction_id,inplace=True) # where there is no cap jurisdiction id 
print("\n   Dwelling Units 2015: {:,}".format(int(du.du.sum())))
print("\n   Dwelling Units 2017: {:,}".format(int(du.du_2017.sum())))

In [ ]:
du_sql = '''
    SELECT parcel_id, mgra_id, luz_id, p.jurisdiction_id, cap_jurisdiction_id, j.name, du_2015 as du, du_2017, capacity_2
        FROM urbansim.parcel p
        LEFT JOIN urbansim.ref.jurisdiction j on p.cap_jurisdiction_id = j.jurisdiction_id
        WHERE du_2017 > 0 or du_2015 > 0'''
du = pd.read_sql(du_sql,mssql_engine)
du.cap_jurisdiction_id.fillna(du.jurisdiction_id,inplace=True) # where there is no cap jurisdiction id 
print("\n   Dwelling Units 2015: {:,}".format(int(du.du.sum())))
print("\n   Dwelling Units 2017: {:,}".format(int(du.du_2017.sum())))

## Get CPAs for city and county

#### complete list for plotting even if no housing stock change, i.e. plot zero for those with no change

In [ ]:
xref_geography_sql = '''
    SELECT mgra_13, cocpa_2016, cicpa_13,cocpa_13, jurisdiction_2016, 
           COALESCE(cocpa_2016,cicpa_13,cocpa_13) as CPAs
      FROM data_cafe.ref.vi_xref_geography_mgra_13'''
xref_geography_df = pd.read_sql(xref_geography_sql, mssql_engine)

In [ ]:
len(xref_geography_df)

#### get unique CPAs

In [ ]:
CPAs = xref_geography_df.loc[~xref_geography_df.CPAs.isnull()].CPAs.unique().tolist()
jurs = xref_geography_df.loc[~xref_geography_df.jurisdiction_2016.isnull()].jurisdiction_2016.unique().tolist()
print(len(CPAs))
print(len(jurs))
Total_city_and_cpa = len(jurs) -2 + len(CPAs)
print(Total_city_and_cpa)

### set "jcid" to CPA where jurisdiction id = 14 or 19

##### simulation output

In [ ]:
# simulation output
units = pd.merge(hs,xref_geography_df,left_on='mgra_id',right_on='mgra_13',how='left')
units.loc[units.cap_jurisdiction_id == 19,'jcid'] = units['cocpa_2016']
units.loc[units.cap_jurisdiction_id == 14,'jcid'] = units['cicpa_13']
units['jcid'].fillna(units['cap_jurisdiction_id'],inplace=True)
print(len(units))
print(len(hs))

##### dwelling units

In [ ]:
# dwelling units
dus = pd.merge(du,xref_geography_df,left_on='mgra_id',right_on='mgra_13',how = 'left')
dus.loc[dus.cap_jurisdiction_id == 19,'jcid'] = dus['cocpa_2016']
dus.loc[dus.cap_jurisdiction_id == 14,'jcid'] = dus['cicpa_13']
dus['jcid'].fillna(dus['cap_jurisdiction_id'],inplace=True)
print(len(dus))
print(len(du))

### Check for nulls in jcid or where jcid equal to 14 or 19 instead of a cpa

##### simulation output

In [ ]:
print(units.loc[units.jcid.isnull()][['parcel_id','jcid','name','cap_jurisdiction_id']])
print(units.loc[units.jcid==14][['parcel_id','jcid','name','cap_jurisdiction_id']])

In [ ]:
units.loc[units.jcid==19][['parcel_id','jcid','name','cap_jurisdiction_id','jurisdiction_id','jurisdiction_2016',\
                           'mgra_id','mgra_13',\
                           'cocpa_2016','cicpa_13','cocpa_13','hs_change']]

In [ ]:
units.head()

##### dwelling units

In [ ]:
print(dus.loc[dus.jcid.isnull()][['parcel_id','jcid','name','cap_jurisdiction_id']])

In [ ]:
dus.loc[dus.jcid==14][['parcel_id','jcid','name','cap_jurisdiction_id','jurisdiction_id','jurisdiction_2016',\
                           'mgra_id','mgra_13',\
                           'cocpa_2016','cicpa_13','cocpa_13']]

In [ ]:
dus.loc[dus.jcid==19][['parcel_id','jcid','name','cap_jurisdiction_id','jurisdiction_id','jurisdiction_2016',\
                           'mgra_id','mgra_13',\
                           'cocpa_2016','cicpa_13','cocpa_13']]

### Manually assign CPA to mgra

##### sql query to find nearest CPA if necessary

##### simulation output CPA 1909 to mgra 19415 & 18831

In [ ]:
units.loc[units.mgra_id==19415,'jcid'] = 1909
units.loc[units.mgra_id==18831,'jcid'] = 1909

##### dwelling units CPA 1909 to mgra 19415 & 18831, jur 13 to mgra 11514 & jur 3 to mgra 7521

In [ ]:
# manually assign CPA or jurisdiction to mgra 19415,11514,18831
dus.loc[dus.mgra_id==19415,'jcid'] = 1909
dus.loc[dus.mgra_id==18831,'jcid'] = 1909
dus.loc[dus.mgra_id==11514.0,'jcid'] = 13
dus.loc[dus.mgra_id==7521.0,'jcid'] = 3
# dus.loc[dus.mgra_13==7259,'jcid'] = 1439 

###  Add CPAs where there was no unit change or dwelling units to complete dataset 

##### simulation output

In [ ]:
units.jcid = units.jcid.astype(int)
units.parcel_id = units.parcel_id.astype(int)
units.set_index('jcid',inplace=True) # necessary for adding missing CPAs
jcids = units.index.unique().tolist()
CPAs_no_unit_change = np.setdiff1d(CPAs,jcids).tolist()
CPAs_no_unit_change

In [ ]:
# need jcid as index here
for cpa in CPAs_no_unit_change:
    units.loc[cpa] = np.nan
units.reset_index(inplace=True) # reset index after assigning CPAs
units.fillna(0,inplace=True)
len(units.jcid.unique())

set simulation year to '2017' for CPAs that had no housing unit change to complete dataset

In [ ]:
# set year to 2017 for 9 CPAs with no data
units.loc[units.year_simulation==0,'year_simulation'] = 2017
units.year_simulation = units.year_simulation.astype(int)
units.loc[units.jcid.isin(CPAs_no_unit_change)][['parcel_id','jcid','cap_jurisdiction_id','hs_change',\
                          'year_simulation']]

##### dwelling units

In [ ]:
dus.jcid = dus.jcid.astype(int)
dus.parcel_id = dus.parcel_id.astype(int)
dus.set_index('jcid',inplace=True) # necessary for adding missing CPAs
jcids = dus.index.unique().tolist()
CPAs_no_unit_change = np.setdiff1d(CPAs,jcids).tolist()
CPAs_no_unit_change

In [ ]:
# need jcid as index here
for cpa in CPAs_no_unit_change:
    dus.loc[cpa] = np.nan
dus.reset_index(inplace=True) # reset index after assigning CPAs
dus.fillna(0,inplace=True)
len(dus.jcid.unique())

## Add cpa name

In [ ]:
cocpa_names_sql = '''
    SELECT zone as cocpa_id, name as cocpa
    FROM data_cafe.ref.geography_zone WHERE geography_type_id = 20'''
cocpa_names = pd.read_sql(cocpa_names_sql, mssql_engine)
cicpa_names_sql = '''
    SELECT zone as cicpa_id, name as cicpa
    FROM data_cafe.ref.geography_zone WHERE geography_type_id = 15'''
cicpa_names = pd.read_sql(cicpa_names_sql, mssql_engine)
luz_names_sql = '''
    SELECT zone as luz_id, name as luz
    FROM data_cafe.ref.geography_zone WHERE geography_type_id = 64'''
luz_names = pd.read_sql(luz_names_sql, mssql_engine)    
jurisdictions_names_sql = '''
    SELECT zone as jurisdiction_id_data_cafe, name as jurisdiction
    FROM data_cafe.ref.geography_zone WHERE geography_type_id = 150'''
jur_name_df = pd.read_sql(jurisdictions_names_sql, mssql_engine)

In [ ]:
units = pd.merge(units,cocpa_names,left_on='jcid',right_on='cocpa_id',how = 'left')
units = pd.merge(units,cicpa_names,left_on='jcid',right_on='cicpa_id',how = 'left')
units = pd.merge(units,jur_name_df,left_on='jurisdiction_2016',right_on='jurisdiction_id_data_cafe',how = 'left')

In [ ]:
units['jcname'] = units['name']
units.loc[units.jcid>=1900,'jcname'] = units['cocpa']
units.loc[(units.jcid>=1400) & (units.jcid<1900),'jcname'] = units['cicpa']
units.loc[units.jcname==0,'jcname'] = units['jurisdiction']

In [ ]:
# check that there are no nulls values (should equal 103)
len(units.jcname.unique())

#### check results again to match jurisdiction feedback page

In [ ]:
change_df = pd.DataFrame({'chg': units.groupby(['jcid','jcname']).
                               hs_change.sum()}).reset_index()
change_df['jcid'] =change_df['jcid'].astype(int)
change_df.set_index('jcid',inplace=True)
# change_df

## Get SR13 results

In [ ]:
sr13_sql = '''
select x.mgra, sum([hs]) AS hs, increment, city, cpa, x.luz as luz_id,site
from [regional_forecast].[sr13_final].[capacity] x
join [regional_forecast].[sr13_final].[mgra13] y
on x.mgra = y.mgra
where scenario = 0 
group by x.mgra, site, increment, y.city,y.cpa,x.luz
order by x.mgra, increment'''
sr13_df = pd.read_sql(sr13_sql, mssql_engine)
# count results
nmgra = int(len(sr13_df.mgra.unique()))
nincrements = int(len(sr13_df.increment.unique()))
lendf = int(len(sr13_df))
duplicated_df = sr13_df[sr13_df.duplicated(subset=['mgra','increment'], keep="first")]
numdup = int(len(duplicated_df))
mi = nmgra*nincrements
nodups = lendf - numdup
print("\n   Number of increments: {:,}".format(nincrements))
print("\n   Number of mgras: {:,}".format(nmgra))
print("\n        increments * mgras: {:,}".format(mi))
print("\nsr13 dataframe length: {:,}".format(lendf))
print("\n   Number of duplicates: {:,}  (mgra w site id and not site id)".format(numdup))

## Get CPAs for city and county for sr13

In [ ]:
sr13_df = pd.merge(sr13_df,jur_name_df[['jurisdiction_id_data_cafe','jurisdiction']],left_on='city',\
                   right_on='jurisdiction_id_data_cafe')
sr13_df = pd.merge(sr13_df,cocpa_names[['cocpa_id','cocpa']],left_on='cpa',right_on='cocpa_id', how = 'outer')
sr13_df = pd.merge(sr13_df,cicpa_names[['cicpa_id','cicpa']],left_on='cpa',right_on='cicpa_id',how = 'outer')
sr13_df = pd.merge(sr13_df,luz_names[['luz_id','luz']],left_on='luz_id',right_on='luz_id')
sr13_df['jur_or_cpa_name'] = sr13_df['cocpa']
sr13_df['jur_or_cpa_name'].fillna(sr13_df['cicpa'],inplace=True)
sr13_df['jur_or_cpa_name'].fillna(sr13_df['jurisdiction'],inplace=True)
sr13_df['jur_or_cpa_id'] = sr13_df['cocpa_id']
sr13_df['jur_or_cpa_id'].fillna(sr13_df['cicpa_id'],inplace=True)
sr13_df['jur_or_cpa_id'].fillna(sr13_df['jurisdiction_id_data_cafe'],inplace=True)
sr13_df['jur_or_cpa_id'] = sr13_df['jur_or_cpa_id'].astype(int)
sr13_df.drop(['jurisdiction_id_data_cafe', 'cocpa_id', 'cicpa_id', 'cocpa', 'cicpa'], axis=1,inplace=True)
# sr13_df.to_csv('test.csv')

In [ ]:
sr13_df.head()

In [ ]:
len(sr13_df['jur_or_cpa_id'].unique())

## sum dwelling units by jursidictions and CPAs (n=103)

In [ ]:
# dus.head()

In [ ]:
du_sr14_geo_df = dus.groupby(['jcid'])[["du","du_2017"]].sum()
du_sr14_geo_df['du_2017'] = du_sr14_geo_df['du_2017'].astype(int)
du_sr14_geo_df['du'] = du_sr14_geo_df['du'].astype(int)
print("\n Total residential dwelling units 2015: {:,}".format(int(du_sr14_geo_df.du.sum())))
print("\n Total residential dwelling units 2017: {:,}".format(int(du_sr14_geo_df.du_2017.sum())))
print("\n Total number of jurisdictions and cpas: {:,}\n".format(len(du_sr14_geo_df.index.unique())))

In [ ]:
du_sr14_geo_df.head()

## sum hs change in simulation by jursidictions and CPAs (n=103)

In [ ]:
sr14_geo_df = pd.DataFrame({'hs_sum': units.groupby(['jcid','jcname','year_simulation']).\
                            hs_change.sum()}).reset_index()
sr14_geo_df.rename(columns = {'jcname':'geo'},inplace=True)
# sr14_geo_df.rename(columns = {'year_simulation':'increment'},inplace=True)
sr14_geo_df.sort_values(by='jcid',inplace=True)
sr14_geo_df.set_index('jcid',inplace=True)
sr14_geo_df['hs_sum'] = sr14_geo_df['hs_sum'].astype(int)
sr14_geo_df['year_simulation'] = sr14_geo_df['year_simulation'].astype(int)
print("\n Total housing unit change after groupby: {:,}".format(int(sr14_geo_df.hs_sum.sum())))
print("\n Total number of jurisdictions and cpas: {:,}\n".format(len(sr14_geo_df.index.unique())))

In [ ]:
sr14_geo_df.head()

## sched dev totals - sum simulation output by source (fill NA with "0")

In [ ]:
sr14_source = pd.DataFrame({'hs_sum': units.groupby(['source','jcid','jcname']).\
                            hs_change.sum()}).reset_index()
sr14_source.rename(columns = {'jcname':'geo'},inplace=True)
sr14_source.sort_values(by='jcid',inplace=True)
sr14_source.set_index('jcid',inplace=True)
sr14_source['hs_sum'] = sr14_source['hs_sum'].astype(int)
print("\n Total housing unit change after groupby: {:,}".format(int(sr14_source.hs_sum.sum())))
print("\n Total number of jurisdictions and cpas: {:,}\n".format(len(sr14_source.index.unique())))

In [ ]:
idx = range(1,4)
sr14_source.set_index(['geo','source'],append=True,inplace=True)
sr14_source = sr14_source.unstack(['jcid','geo'])
# sr14_source = sr14_source.reindex(idx, fill_value=0)
sr14_source.fillna(0,inplace=True)
sr14_source = sr14_source.stack(['jcid','geo'])
sr14_source.reset_index(inplace=True)
sr14_source.set_index('jcid',inplace=True)

##### sched dev totals

In [ ]:
sr14_source1 =  sr14_source.loc[sr14_source.source==1.0].copy()
sr14_source1.head(5)
print(sr14_source1.hs_sum.sum())

### Get SR13 sched dev totals by jur and cpa

In [ ]:
sr13_sched_dev_sql = '''
SELECT x.mgra,city, cpa, x.luz as luz_id,site, sum(siteSF+siteMF+siteMH) AS sched_dev
FROM [regional_forecast].[sr13_final].[capacity] x
join [regional_forecast].[sr13_final].[mgra13] y
on x.mgra = y.mgra
where scenario = 0 and increment = 2050 and site != 0 and (siteSF + siteMF + siteMH) > 0 
group by x.mgra, site, increment, y.city,y.cpa,x.luz
order by city,site'''
sr13_sched_dev = pd.read_sql(sr13_sched_dev_sql, mssql_engine)

In [ ]:
sr13_sched_dev.sched_dev.sum()

In [ ]:
sr13_sched_dev = pd.merge(sr13_sched_dev,jur_name_df[['jurisdiction_id_data_cafe','jurisdiction']],\
                          left_on='city',right_on='jurisdiction_id_data_cafe',how='outer')
sr13_sched_dev = pd.merge(sr13_sched_dev,cocpa_names[['cocpa_id','cocpa']],left_on='cpa',right_on='cocpa_id', how = 'outer')
sr13_sched_dev = pd.merge(sr13_sched_dev,cicpa_names[['cicpa_id','cicpa']],left_on='cpa',right_on='cicpa_id',how = 'outer')
# sr13_sched_dev = pd.merge(sr13_sched_dev,luz_names[['luz_id','luz']],left_on='luz_id',right_on='luz_id',how='outer')
sr13_sched_dev['jur_or_cpa_name'] = sr13_sched_dev['cocpa']
sr13_sched_dev['jur_or_cpa_name'].fillna(sr13_sched_dev['cicpa'],inplace=True)
sr13_sched_dev['jur_or_cpa_name'].fillna(sr13_sched_dev['jurisdiction'],inplace=True)
sr13_sched_dev['jur_or_cpa_id'] = sr13_sched_dev['cocpa_id']
sr13_sched_dev['jur_or_cpa_id'].fillna(sr13_sched_dev['cicpa_id'],inplace=True)
sr13_sched_dev['jur_or_cpa_id'].fillna(sr13_sched_dev['jurisdiction_id_data_cafe'],inplace=True)
# sr13_sched_dev['jur_or_cpa_id'] = sr13_sched_dev['jur_or_cpa_id'].astype(int)
sr13_sched_dev.drop(['jurisdiction_id_data_cafe', 'cocpa_id', 'cicpa_id', 'cocpa', 'cicpa'], axis=1,inplace=True)
sr13_sched_dev.rename(columns = {'jur_or_cpa_name':'geo'},inplace=True)

In [ ]:
sr13_sched_dev.loc[sr13_sched_dev.jur_or_cpa_id ==8]

In [ ]:
sr13_sched_dev.loc[sr13_sched_dev.jur_or_cpa_id.isnull()]

In [ ]:
sr13_sched_dev.fillna(0,inplace=True)

In [ ]:
sr13_sched_dev['jur_or_cpa_id'] = sr13_sched_dev['jur_or_cpa_id'].astype(int)

In [ ]:
sr13_sched_dev_totals = pd.DataFrame({'sched_dev_sum': sr13_sched_dev.
                                            groupby(["jur_or_cpa_id",'geo']).
                                 sched_dev.sum()}).reset_index()

In [ ]:
sr13_sched_dev_totals.sched_dev_sum.sum()

In [ ]:
sr13_sched_dev_totals.head()

## Fill in "0" for units for "missing" simulation years (for plotting) (e.g. Del Mar)

In [ ]:
# Del Mar example
del_mar_before = sr14_geo_df.loc[4].sort_values(by='year_simulation')
# del_mar_before.head()
del_mar_before.plot(x='year_simulation',y='hs_sum',style='.-',title='NULL values in Del Mar Housing Unit Change')

In [ ]:
# del_mar_before

In [ ]:
idx = range(2017,2051)
sr14_geo_df.set_index(['geo','year_simulation'],append=True,inplace=True)
sr14_geo_df = sr14_geo_df.unstack(['jcid','geo'])
sr14_geo_df = sr14_geo_df.reindex(idx, fill_value=0)
sr14_geo_df.fillna(0,inplace=True)
sr14_geo_df = sr14_geo_df.stack(['jcid','geo'])
sr14_geo_df.reset_index(inplace=True)
sr14_geo_df.set_index('jcid',inplace=True)

In [ ]:
del_mar_after = sr14_geo_df.loc[4].sort_values(by='year_simulation')
del_mar_after.plot(x='year_simulation',y='hs_sum',style='.-',title='Replace Null with Zeroes Del Mar Housing Unit Change')

In [ ]:
len(sr14_geo_df.geo.unique())

In [ ]:
len(du_sr14_geo_df)

In [ ]:
len(sr14_geo_df.year_simulation.unique())

In [ ]:
len(sr14_geo_df.year_simulation.unique()) * len(sr14_geo_df.geo.unique())

In [ ]:
len(sr14_geo_df)

In [ ]:
sr14_geo_df.head()

## Sum units from output of simulation over five year increments

In [ ]:
bins = range(2015,2055,5)
names = [str(x) for x in range(2020,2055,5)]
sr14_geo_df['increment'] = pd.cut(sr14_geo_df.year_simulation, bins, labels=names)

In [ ]:
sr14_increment = pd.DataFrame({'hs_increment': sr14_geo_df.
                                            groupby(["increment","jcid","geo"]).
                                 hs_sum.sum()}).reset_index()
# sr14_increment.set_index('jcid',inplace=True)

In [ ]:
sr14_increment.head()

In [ ]:
sr14_increment.increment.unique()

In [ ]:
sr14_increment.increment = sr14_increment.increment.astype(int)

## Cumulative sum units added by increment

In [ ]:
sr14_increment['hs_cumulative'] = sr14_increment.groupby(['geo'])['hs_increment'].apply(lambda x: x.cumsum())
sr14_increment.set_index('jcid',inplace=True)

In [ ]:
sr14_increment.loc[sr14_increment.geo=='Carlsbad']

## Join simulation output with existing dwelling units

In [ ]:
len(sr14_increment)

In [ ]:
len(du_sr14_geo_df)

In [ ]:
sr14 = sr14_increment.join(du_sr14_geo_df)

In [ ]:
sr14.head()

In [ ]:
sr14['hs'] = sr14['hs_cumulative'] + sr14['du_2017']

In [ ]:
sr14.loc[1].sort_values(by='increment')

In [ ]:
results2050 = sr14.loc[sr14.increment==2050].copy()

In [ ]:
results2050.head(2)

In [ ]:
results2050['Housing Units 2050'] = results2050['hs'].astype(int)

In [ ]:
results2050['Housing Units 2017'] = results2050['du_2017'].astype(int)

In [ ]:
results2050['Housing Units Change'] = results2050['hs_cumulative'].astype(int)

In [ ]:
results2050['Name'] = results2050['geo']

In [ ]:
results2050['id'] = results2050.index
results2050['City'] = results2050.Name

In [ ]:
results2050.loc[results2050.index.isin(range(1399,1501)),'id'] = 14
results2050.loc[results2050.index.isin(range(1899,2001)),'id'] = 19

In [ ]:
results2050.loc[results2050.index.isin(range(1399,1501)),'City'] = 'San Diego'
results2050.loc[results2050.index.isin(range(1899,2001)),'City'] = 'Unincorporated'

In [ ]:
juroutput_chg = pd.DataFrame({'Total Unit Change':  results2050.groupby(['id','City']).\
                          hs_cumulative.sum()}).reset_index()
juroutput_chg.set_index('id',inplace=True)
juroutput_chg['Total Unit Change'] = juroutput_chg['Total Unit Change'].astype(int)

In [ ]:
juroutput_chg3 = pd.DataFrame({'Units 2050':  results2050.groupby(['id','City']).\
                          hs.sum()}).reset_index()
juroutput_chg3.set_index('id',inplace=True)
juroutput_chg3['Units 2050'] = juroutput_chg3['Units 2050'].astype(int)

In [ ]:
juroutput_chg2 = pd.DataFrame({'Units 2017':  results2050.groupby(['id','City']).\
                          du_2017.sum()}).reset_index()
juroutput_chg2.set_index('id',inplace=True)

juroutput_chg2['Units 2050'] = juroutput_chg3['Units 2050']
juroutput_chg2['Total Unit Change'] = juroutput_chg['Total Unit Change']

In [ ]:
juroutput_chg2.loc['Total']= juroutput_chg2.sum()

In [ ]:
juroutput_chg2.loc[juroutput_chg2.index=='Total','City'] = 'Region'

In [ ]:
# juroutput_chg2

In [ ]:
sr13_df.head()

In [ ]:
output_sr13_df = pd.DataFrame({'hs_sum': sr13_df.groupby(['city','jurisdiction','increment']).
                               hs.sum()}).reset_index()

output_sr13_df.head()

In [ ]:
results2050 = sr14.loc[sr14.increment==2050].copy()

In [ ]:
#output2050 = results2050[['Name','Housing Units 2017','Housing Units 2050','Housing Units Change']].copy()

In [ ]:
#output2050.loc['Total']= output2050.sum()

In [ ]:
#output2050.loc[output2050.index=='Total','Name'] = 'Region'

In [ ]:
#output2050['jur'] = output2050.index

In [ ]:
# juroutput

In [ ]:
# output2050.loc[output2050.index.isin(range(1399,1501))]

In [ ]:
# output2050.loc[output2050.index.isin(range(0,19))]

## Add increment 2016 with units added equal to zero for baseline du (for plotting)

In [ ]:
start_year = sr14_geo_df.loc[sr14_geo_df.year_simulation==2017].copy()
len(start_year)
start_year['increment'] = 2015
start_year = start_year.join(du_sr14_geo_df)
start_year['hs'] = start_year['du']
start_year['year_simulation'] = 'baseline'
sr14 = pd.concat([sr14,start_year])

In [ ]:
start_year['increment'] = 2017
start_year['hs'] = start_year['du_2017']
sr14 = pd.concat([sr14,start_year])

In [ ]:
# sr14.loc[1420].sort_values(by='increment')

## SR13 aggregrate

####  mgra level housing to cpa and jurisdiction and group by increment and sum

In [ ]:
sr13_geo_df = pd.DataFrame({'hs_sum': sr13_df.groupby(['jur_or_cpa_name','jur_or_cpa_id','increment']).
                               hs.sum()}).reset_index()
sr13_geo_df.rename(columns = {'jur_or_cpa_name':'geo'},inplace=True)
sr13_geo_df.sort_values(by='jur_or_cpa_id',inplace=True)
sr13_geo_df.head()

In [ ]:
print("\nNum of geographies (jurisdiction plus cpa) = {:,}\n".format(int(len(sr13_geo_df.loc[sr13_geo_df.increment==2015]))))

In [ ]:
sr13_geo_df.loc[sr13_geo_df.geo=='Escondido'].sort_values(by='increment')

##  sr13 pivot so each jurisdiction or cpa is column and rows are increments

In [ ]:
sr13_geo_df_pivot = sr13_geo_df.pivot\
(index='increment', columns='geo', values='hs_sum').\
reset_index().rename_axis(None, axis=1)
sr13_geo_df_pivot.set_index('increment',inplace=True)
sr13_geo_df_pivot

##  sr14 pivot so each jurisdiction or cpa is column and rows are increments

In [ ]:
sr14_geo_df_pivot = sr14.pivot\
(index='increment', columns='geo', values='hs').\
reset_index().rename_axis(None, axis=1)
sr14_geo_df_pivot.set_index('increment',inplace=True)
sr14_geo_df_pivot.fillna(0,inplace=True)
sr14_geo_df_pivot

####  sr13 calculate total change by region and jurisdiction and cpa

In [ ]:
total_diff = sr13_geo_df_pivot.loc[[2012,2050],:]
differences = total_diff.diff().loc[[2050]]
differences.rename(index={2050: 'total_change'},inplace=True)
totalchange = pd.DataFrame(differences.sum(axis=0))
totalchange.rename(columns={0: 'total_change'},inplace=True)
sr13_totalchange = pd.DataFrame(differences.sum(axis=0))
sr13_totalchange.rename(columns={0: 'total_change'},inplace=True)
print("\nTotal Units added sr13: {:,}".format(int(totalchange.total_change.sum())))

In [ ]:
geolist = sr14_geo_df.geo.unique().tolist()

In [ ]:
l = []
for jur in (geolist):
    s = str(jur)
    l.append(s)

In [ ]:
#print '[%s]' % ', '.join(map(str, l))

In [ ]:
geolist[1:5]

In [ ]:
jur = geolist[4]

In [ ]:
jur

In [ ]:
scheddevlabel_sr13 = '(' + str(int( sr13_sched_dev_totals.loc[ sr13_sched_dev_totals['geo']==jur].iloc[0]['sched_dev_sum'])) + ' sched dev)'    
scheddevlabel_sr13

# plot

### directory for plots

In [ ]:
#print os.getcwd()

In [ ]:
dirname = os.path.join(os.getcwd(),'plots')#,datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S'))
                       
                       # 'run_'+str(run_id) + '_city_cpa')
                         
try:
    os.stat(dirname)
except:
    os.mkdir(dirname)  

In [ ]:
# print dirname

In [ ]:
# sr13_geo_df_pivot.columns.tolist()
# sr13_geo_df_pivot[[jur]]
# sr14_geo_df_pivot[[jur]]
len(sr14_geo_df_pivot.columns.tolist())

In [ ]:
sr14_geo_df_pivot.head()

In [ ]:
# sr14_geo_df_pivot.to_csv('test.csv')

In [ ]:
# scheddevlabel_sr13

In [ ]:
# '(' + str(int( sr13_sched_dev_totals.loc[ sr13_sched_dev_totals['geo']==jur].iloc[0]['sched_dev_sum'])) + ' sched dev)'    

In [ ]:
sr14_geo_df.head()

In [ ]:
jur

In [ ]:
geolist[0:1]

#### plot figures as png

In [ ]:
### COPY
for j, jur in enumerate(geolist):
        fig = plt.figure()
        ax = fig.add_subplot(111)
        chg = int(sr14_increment.loc[(sr14_increment.increment==2050) & (sr14_increment.geo==jur)].hs_cumulative)
        plotlabelsr13 = 'sr13 '
        plotlabelsr14 = 'sr14 '
        scheddevlabel_sr13 = '(' + str(int( sr13_sched_dev_totals.loc[ sr13_sched_dev_totals['geo']==jur].iloc[0]['sched_dev_sum'])) + ' sched dev)'    
        scheddevlabel = '(' + str(int(sr14_source1.loc[sr14_source1['geo']==jur].iloc[0]['hs_sum'])) + ' sched dev)'
        sr14_increments_int = sr14_geo_df_pivot[[jur]].reset_index().increment.tolist()
        sr13_increments_int = sr13_geo_df_pivot[[jur]].reset_index().increment.tolist()
        
        
        sr14 = ax.plot(sr14_increments_int,
                       sr14_geo_df_pivot[[jur]].reset_index()[jur].tolist(),'b-o')
        sr13 = plt.plot(sr13_increments_int,
                        sr13_geo_df_pivot[[jur]].reset_index()[jur].tolist(),'r-o')
          
        city_cpa_id = str(sr14_geo_df.loc[sr14_geo_df['geo']==jur].index.values[0])  
       
        # legendtitle = jur + ' (' +  city_cpa_id + ')'

                
        # create blank rectangle
        extra = Rectangle((0, 0), 1, 1, fc="w", fill=False, edgecolor='none', linewidth=0)
        ax.legend([sr14[0],sr13[0]],\
                  (plotlabelsr14,plotlabelsr13))
        # extra2 = Rectangle((0, 0), 1, 1, fc="w", fill=False, edgecolor='none', linewidth=0)
        # extra3 = Rectangle((0, 0), 1, 1, fc="w", fill=False, edgecolor='none', linewidth=0)
        # ax.legend([extra, sr13[0],extra2,sr14[0],extra3], ("Total Chg", plotlabelsr13,scheddevlabel_sr13, plotlabelsr14,scheddevlabel))
        # plt.ylabel('Housing stock')
        # plt.xlabel('Year')
        # plt.title('Series 13 and DRAFT Series 14 Comparison\n' + jur + ' (id=' +  city_cpa_id + ')')
        ptitle = 'DRAFT Series 14 and Series 13 Comparison\n' + jur + ' (id=' +  str(city_cpa_id) + ')'
        ax.set_title(ptitle,fontsize= 14)
        ax.set_xlabel("Year",size=14)
        ax.set_ylabel("Housing stock",size=14)
        # ptitle = 'DRAFT Cumulative Housing Units Added By Capacity Type\n' + jur + ' (id=' +  str(city_cpa_id) + ')'
        jur_ = jur.replace(":", "_") # colon not allowed in windows filename
        plotname = 'plots' + '//' + str(run_id) + '_city_cpa_' + city_cpa_id + '_' + jur_ + '.png'
        # plotname = dirname + '//' + str(run_id) + '_city_cpa_' + city_cpa_id + '_' + jur_ + '.png'

        fig = plt.gcf()
        fig.set_size_inches(6, 4.5)
        fig.savefig(plotname, bbox_inches='tight',dpi=200)
        plt.show()
        plt.close(fig)
        
        
        # ptitle = 'DRAFT Cumulative Housing Units Added By Capacity Type\n' + jur + ' (id=' +  str(city_cpa_id) + ')'
        # ax.set_ylabel("housing units added",size=14)
        # ax.set_title(ptitle,size=14)

In [ ]:
# blank

In [ ]:

for j, jur in enumerate(geolist):
    if jur == 'blank':
        img = np.zeros([100,100,3],dtype=np.uint8)
        img.fill(255)
        # plotname = 'out/luz4/' + str(j+1) + '_luz.png'
        # imsave( plotname,img)
    else:
        fig = plt.figure()
        ax = fig.add_subplot(111)
        
        chg = int(sr14_increment.loc[(sr14_increment.increment==2050) & (sr14_increment.geo==jur)].hs_cumulative)
        
        plotlabelsr13 = 'sr13: ' +  ' ' + str(int(totalchange.loc[jur][0]))
        
        plotlabelsr14 = 'sr14: '+ ' ' + str(chg)  # + '\n(sched dev = ' + \
              # str(int(sr14_source1.loc[sr14_source1['geo']==jur].iloc[0]['hs_sum'])) + ')'
            
        scheddevlabel_sr13 = '(' + str(int( sr13_sched_dev_totals.loc[ sr13_sched_dev_totals['geo']==jur].iloc[0]['sched_dev_sum'])) + ' sched dev)'    
        scheddevlabel = '(' + str(int(sr14_source1.loc[sr14_source1['geo']==jur].iloc[0]['hs_sum'])) + ' sched dev)'
        
        sr14_increments_int = sr14_geo_df_pivot[[jur]].reset_index().increment.tolist()
        
        
        sr14 = ax.plot(sr14_increments_int,
                       sr14_geo_df_pivot[[jur]].reset_index()[jur].tolist(),'b-o', label=plotlabelsr14)
    
        sr13_increments_int = sr13_geo_df_pivot[[jur]].reset_index().increment.tolist()
        
    
        sr13 = plt.plot(sr13_increments_int,
                        sr13_geo_df_pivot[[jur]].reset_index()[jur].tolist(),'r-o', label=plotlabelsr13)
          
        city_cpa_id = str(sr14_geo_df.loc[sr14_geo_df['geo']==jur].index.values[0])    
        legendtitle = jur + ' (' +  city_cpa_id + ')'

                
        # create blank rectangle
        extra = Rectangle((0, 0), 1, 1, fc="w", fill=False, edgecolor='none', linewidth=0)
        extra2 = Rectangle((0, 0), 1, 1, fc="w", fill=False, edgecolor='none', linewidth=0)
        extra3 = Rectangle((0, 0), 1, 1, fc="w", fill=False, edgecolor='none', linewidth=0)
        ax.legend([extra, sr13[0],extra2,sr14[0],extra3], ("Total Chg", plotlabelsr13,scheddevlabel_sr13, plotlabelsr14,scheddevlabel))
        plt.ylabel('Housing stock')
        plt.xlabel('Increment')
        plt.title('Series 13 and DRAFT Series 14 Comparison\n' + legendtitle)
        jur_ = jur.replace(":", "_") # colon not allowed in windows filename
        plotname = dirname + '//' + str(run_id) + '_city_cpa_' + city_cpa_id + '_' + jur_ + '.png'
        
        fig = plt.gcf()
        fig.set_size_inches(6, 4.5)
        fig.savefig(plotname, bbox_inches='tight',dpi=200)
        #plt.show()
        plt.close(fig)

#  AS PDF instead of png

units_added_by_source.plot(style='.-',stacked=True,kind='bar',ax=axes.flat[2],\
                           figsize=(10,15),color=['red','purple','green','sandybrown'])

# QC data

In [ ]:
units.columns

In [ ]:
units[['jcid','name','parcel_id','hs_change','source','year_simulation']].head()

In [ ]:
j1414 = units[['jcid','name','parcel_id','hs_change','source','year_simulation']].loc[units.jcid==1414]

In [ ]:
j1414.sort_values(by='year_simulation',inplace=True)

In [ ]:
j1414.hs_change.sum()

In [ ]:
# j1414

In [ ]:
# get max run id from urbansim
run_id_sql = '''
SELECT max(run_id)
  FROM [urbansim].[urbansim].[urbansim_lite_output]
'''
run_id_df = pd.read_sql(run_id_sql, mssql_engine)
run_id = int(run_id_df.values)
print("\nRun id : {:,}".format(run_id))

hs_change_sql = '''
    SELECT o.parcel_id, j.name,  p.cap_jurisdiction_id, p.jurisdiction_id, p.mgra_id, p.luz_id,
    unit_change as hs_change, source, capacity_type,year_simulation
      FROM urbansim.urbansim.urbansim_lite_output o 
      JOIN urbansim.urbansim.parcel p on p.parcel_id = o.parcel_id
      JOIN urbansim.ref.jurisdiction j on p.cap_jurisdiction_id = j.jurisdiction_id
     WHERE run_id =  %s
  ORDER BY j.name,p.jurisdiction_id, year_simulation'''
hs_change_sql = hs_change_sql % run_id
hs = pd.read_sql(hs_change_sql,mssql_engine)
units_added = int(hs.hs_change.sum())

print("\nUnits added: {:,} ".\
      format(units_added))

In [ ]:
hs.head()

In [ ]:
# simulation output
units = pd.merge(hs,xref_geography_df,left_on='mgra_id',right_on='mgra_13',how='left')
units.loc[units.cap_jurisdiction_id == 19,'jcid'] = units['cocpa_2016']
units.loc[units.cap_jurisdiction_id == 14,'jcid'] = units['cicpa_13']
units['jcid'].fillna(units['cap_jurisdiction_id'],inplace=True)
# manually assign CPA to mgra 19415, 18831
units.loc[units.mgra_id==19415,'jcid'] = 1909
units.loc[units.mgra_id==18831,'jcid'] = 1909
print(units.loc[units.jcid.isnull()])
print(units.loc[units.jcid==14])
print(units.loc[units.jcid==19])
units.parcel_id = units.parcel_id.astype(int)
units.jcid = units.jcid.astype(int)
units.set_index('jcid',inplace=True)

In [ ]:
units.head()

In [ ]:
jcids = units.index.unique().tolist()
CPAs_no_unit_change = np.setdiff1d(CPAs,jcids).tolist()
print(CPAs_no_unit_change)
for cpa in CPAs_no_unit_change:
    units.loc[cpa] = np.nan

In [ ]:
units.reset_index(inplace=True)
print(len(units.jcid.unique()))
units.fillna(0,inplace=True)
units = pd.merge(units,cocpa_names,left_on='jcid',right_on='cocpa_id',how = 'left')
units = pd.merge(units,cicpa_names,left_on='jcid',right_on='cicpa_id',how = 'left')

In [ ]:
jur_name_df.head()

In [ ]:
units = pd.merge(units,jur_name_df,left_on='jurisdiction_2016',right_on='jurisdiction_id_data_cafe',how = 'left')
units['jcname'] = units['name']

In [ ]:
units.loc[units.jcid>=1900,'jcname'] = units['cocpa']
units.loc[(units.jcid>=1400) & (units.jcid<1900),'jcname'] = units['cicpa']
units.loc[units.jcname==0,'jcname'] = units['jurisdiction']
print(units.sort_values(by='jcid').jcid.unique())
print(len(units.jcname.unique()))
units.loc[units.year_simulation==0,'year_simulation'] = 2017

In [ ]:
# units.loc[units.jcname=='Barrio Logan']

In [ ]:
units.head()

In [ ]:
hs2 = units.copy()
hs2.replace('cc', 'sgoa',inplace=True)
hs2.replace('mc', 'sgoa',inplace=True)
hs2.replace('tc', 'sgoa',inplace=True)
hs2.replace('tco', 'sgoa',inplace=True)
hs2.replace('uc', 'sgoa',inplace=True)

In [ ]:
bins = range(2015,2055,5)
names = [str(x) for x in range(2020,2055,5)]
hs2['increment'] = pd.cut(hs2.year_simulation, bins, labels=names)

In [ ]:
hs2.head()

In [ ]:

# "jcid","geo"
units_added_by_capacity_type_and_yr = pd.DataFrame({'units_by_type': hs2.
                                          groupby(["jcid","jcname","increment","capacity_type"])
                                          .hs_change.sum()}).reset_index()

In [ ]:
geolist = units_added_by_capacity_type_and_yr.jcname.unique().tolist()

In [ ]:
# geolist

In [ ]:
units_added_by_capacity_type_and_yr.set_index(['jcid','jcname','increment','capacity_type'],inplace=True)
units_added_by_capacity_type_and_yr = units_added_by_capacity_type_and_yr.unstack(['jcid','jcname'])
units_added_by_capacity_type_and_yr.fillna(0,inplace=True)
units_added_by_capacity_type_and_yr = units_added_by_capacity_type_and_yr.stack(['jcid','jcname'])
units_added_by_capacity_type_and_yr.reset_index(inplace=True)

In [ ]:
# for tablular data
assigned_parcel_sql = '''
SELECT  a.parcel_id, cap_jurisdiction_id, jurisdiction_id, a.du, a.type,p.mgra_id
   FROM [urbansim].[urbansim].[additional_capacity] a
   JOIN urbansim.parcel p on p.parcel_id = a.parcel_id
  WHERE version_id = %s'''
assigned_parcel_sql = assigned_parcel_sql % versions['additional_capacity_version']
assigned_df = pd.read_sql(assigned_parcel_sql, mssql_engine)
assigned_df.replace('cc', 'sgoa',inplace=True)
assigned_df.replace('mc', 'sgoa',inplace=True)
assigned_df.replace('tc', 'sgoa',inplace=True)
assigned_df.replace('tco', 'sgoa',inplace=True)
assigned_df.replace('uc', 'sgoa',inplace=True)
assigned_df_1 = assigned_df.loc[assigned_df.cap_jurisdiction_id==1]
assigned_capacity_1 = int(assigned_df_1.du.sum())
assigned_capacity_adu = int(assigned_df_1.loc[assigned_df.type=='adu'].du.sum())
assigned_capacity_sgoa = int(assigned_df_1.loc[assigned_df.type=='sgoa'].du.sum())

print("\nCapacity from urbansim.additional_capacity (ADU and SGOAs): {:,}".format(assigned_capacity_1))
print("\nCapacity from urbansim.additional_capacity (ADU): {:,}".format(assigned_capacity_adu))
print("\nCapacity from urbansim.additional_capacity (SGOAs): {:,}".format(assigned_capacity_sgoa))
# assigned_df.head()

In [ ]:
# for tablular data
assigned_df.replace('cc', 'sgoa',inplace=True)
assigned_df.replace('mc', 'sgoa',inplace=True)
assigned_df.replace('tc', 'sgoa',inplace=True)
assigned_df.replace('tco', 'sgoa',inplace=True)
assigned_df.replace('uc', 'sgoa',inplace=True)

In [ ]:
# for tablular data
assignedcpa = pd.merge(assigned_df,xref_geography_df,left_on='mgra_id',right_on='mgra_13',how='left')
assignedcpa.loc[assignedcpa.cap_jurisdiction_id == 19,'jcid'] = assignedcpa['cocpa_2016']
assignedcpa.loc[assignedcpa.cap_jurisdiction_id == 14,'jcid'] = assignedcpa['cicpa_13']
assignedcpa['jcid'].fillna(assignedcpa['cap_jurisdiction_id'],inplace=True)
assignedcpa.loc[assignedcpa.mgra_id==19415,'jcid'] = 1909
assignedcpa.loc[assignedcpa.mgra_id==18831,'jcid'] = 1909
assignedcpa.loc[assignedcpa.mgra_id==11514.0,'jcid'] = 13
assignedcpa.loc[assignedcpa.mgra_id==7521.0,'jcid'] = 3
assignedcpa.loc[assignedcpa.mgra_13==7259,'jcid'] = 1439 

In [ ]:
# for tablular data
sched_dev_parcel_sql = '''
SELECT s.parcel_id, p.jurisdiction_id, p.cap_jurisdiction_id,capacity_3,mgra_id
FROM [urbansim].[urbansim].[scheduled_development_parcel] s
JOIN [urbansim].[urbansim].[parcel] p
ON p.parcel_id = s.parcel_id'''
sched_dev_df = pd.read_sql(sched_dev_parcel_sql, mssql_engine)
sched_dev_df_1 = sched_dev_df.loc[sched_dev_df.cap_jurisdiction_id==1]
sched_dev_capacity_1 = int(sched_dev_df_1.capacity_3.sum())
print("\nScheduled development capacity from urbansim.scheduled_development_parcel : {:,}".\
      format(sched_dev_capacity_1))

In [ ]:
# for tablular data
sunits = pd.merge(sched_dev_df,xref_geography_df,left_on='mgra_id',right_on='mgra_13',how='left')
sunits.loc[sunits.cap_jurisdiction_id == 19,'jcid'] = sunits['cocpa_2016']
sunits.loc[sunits.cap_jurisdiction_id == 14,'jcid'] = sunits['cicpa_13']
sunits['jcid'].fillna(sunits['cap_jurisdiction_id'],inplace=True)
sunits.loc[sunits.mgra_id==19415,'jcid'] = 1909
sunits.loc[sunits.mgra_id==18831,'jcid'] = 1909
sunits.loc[sunits.mgra_id==11514.0,'jcid'] = 13
sunits.loc[sunits.mgra_id==7521.0,'jcid'] = 3
sunits.loc[sunits.mgra_13==7259,'jcid'] = 1439 

In [ ]:
sched_dev_df_1 = sunits.loc[sunits.jcid==1404]
sched_dev_capacity_1 = int(sched_dev_df_1.capacity_3.sum())

In [ ]:
sched_dev_capacity_1

In [ ]:
# for tablular data
parcel_capacity_sql = '''
    SELECT [parcel_id],[jurisdiction_id],[cap_jurisdiction_id],[site_id],mgra_id,
           [du_2015],[du_2017],[capacity_1],[capacity_2]
      FROM [urbansim].[urbansim].[parcel]'''
capacity_df = pd.read_sql(parcel_capacity_sql,mssql_engine)
capacity_df_1 = capacity_df.loc[capacity_df.cap_jurisdiction_id==1]
urbansim_parcel_capacity_1 = int(capacity_df_1.loc[capacity_df_1.site_id.isnull()].capacity_2.sum())
print("\nCapacity from urbansim.parcel where site id is null: {:,}".format(urbansim_parcel_capacity_1))
# 291,989

In [ ]:
# for tablular data
units = pd.merge(capacity_df,xref_geography_df,left_on='mgra_id',right_on='mgra_13',how='left')
units.loc[units.cap_jurisdiction_id == 19,'jcid'] = units['cocpa_2016']
units.loc[units.cap_jurisdiction_id == 14,'jcid'] = units['cicpa_13']
units['jcid'].fillna(units['cap_jurisdiction_id'],inplace=True)
print(len(units))
print(len(capacity_df))

In [ ]:
# for tablular data
# manually assign CPA or jurisdiction to mgra 19415,11514,18831
units.loc[units.mgra_id==19415,'jcid'] = 1909
units.loc[units.mgra_id==18831,'jcid'] = 1909
units.loc[units.mgra_id==11514.0,'jcid'] = 13
units.loc[units.mgra_id==7521.0,'jcid'] = 3
units.loc[units.mgra_13==7259,'jcid'] = 1439 

In [ ]:
# capacity_df_1 = units.loc[units.jcid==city_cpa_id]

In [ ]:
# city_cpa_id

In [ ]:
# QC tabular data
# units.loc[units.jcid==1999]

In [ ]:
# for tabular data
run_id_sql = '''
SELECT max(run_id)
  FROM [urbansim].[urbansim].[urbansim_lite_output]
'''
run_id_df = pd.read_sql(run_id_sql, mssql_engine)
run_id = int(run_id_df.values)
print("\nRun id : {:,}".format(run_id))

hs_change_sql = '''
    SELECT o.parcel_id, j.name,  p.cap_jurisdiction_id, p.jurisdiction_id, p.mgra_id, p.luz_id,
    unit_change as hs_change, source, capacity_type,year_simulation
      FROM urbansim.urbansim.urbansim_lite_output o 
      JOIN urbansim.urbansim.parcel p on p.parcel_id = o.parcel_id
      JOIN urbansim.ref.jurisdiction j on p.cap_jurisdiction_id = j.jurisdiction_id
     WHERE run_id =  %s
  ORDER BY j.name,p.jurisdiction_id, year_simulation'''
hs_change_sql = hs_change_sql % run_id
hs = pd.read_sql(hs_change_sql,mssql_engine)
units_added = int(hs.hs_change.sum())

print("\nUnits added: {:,} ".\
      format(units_added))

In [ ]:
# for tablular data
hscpa = pd.merge(hs,xref_geography_df,left_on='mgra_id',right_on='mgra_13',how='left')
hscpa.loc[hscpa.cap_jurisdiction_id == 19,'jcid'] = hscpa['cocpa_2016']
hscpa.loc[hscpa.cap_jurisdiction_id == 14,'jcid'] = hscpa['cicpa_13']
hscpa['jcid'].fillna(hscpa['cap_jurisdiction_id'],inplace=True)
hscpa.loc[hscpa.mgra_id==19415,'jcid'] = 1909
hscpa.loc[hscpa.mgra_id==18831,'jcid'] = 1909
hscpa.loc[hscpa.mgra_id==11514.0,'jcid'] = 13
hscpa.loc[hscpa.mgra_id==7521.0,'jcid'] = 3
hscpa.loc[hscpa.mgra_13==7259,'jcid'] = 1439 

In [ ]:
hscpa.head()

In [ ]:
dirname = os.path.join(os.getcwd(),'plots/barplots4')#,datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S'))
# 'run_'+str(run_id) + '_city_cpa')
try:
    os.stat(dirname)
except:
    os.mkdir(dirname)  

In [ ]:
geolist[18:19]

# use this for bar plots and tabular data

In [ ]:
for j, jur in enumerate(geolist):
        c = units_added_by_capacity_type_and_yr.loc[units_added_by_capacity_type_and_yr.jcname==jur]
        type_pivot = c.pivot(index='increment', columns='capacity_type', values='units_by_type').\
        reset_index().rename_axis(None, axis=1)
        

        type_pivot.set_index('increment',inplace=True)
        type_pivot.fillna(0,inplace=True)
        type_pivot = type_pivot[['sch', 'jur', 'adu', 'sgoa']]
        type_pivot['sch'] = type_pivot['sch'].cumsum()
        type_pivot['jur'] = type_pivot['jur'].cumsum()
        type_pivot['adu'] = type_pivot['adu'].cumsum()
        type_pivot['sgoa'] = type_pivot['sgoa'].cumsum()
        
        city_cpa_id = c.iloc[0]['jcid']
        
        ax = type_pivot.plot.bar(stacked=True,rot=0)

        ptitle = 'DRAFT Cumulative Housing Units Added By Capacity Type\n' + jur + ' (id=' +  str(city_cpa_id) + ')'
        ax.set_ylabel("Housing units added",size=14)
        ax.set_title(ptitle,size=14)
        ax.set_xlabel("Year",size=14)

        fig = plt.gcf()
        fig.set_size_inches(6, 4.5)
        jur_ = jur.replace(":", "_") # colon not allowed in windows filename
        plotname = dirname + '//' + str(run_id) + '_city_cpa_' + str(city_cpa_id) + '_' + jur_ + 'bar.png'
        fig.savefig(plotname, bbox_inches='tight',dpi=200)
        plt.show()
        plt.close(fig)
        
        # table
        c = units_added_by_capacity_type_and_yr.loc[units_added_by_capacity_type_and_yr.jcname==jur]
        type_pivot = c.pivot(index='increment', columns='capacity_type', values='units_by_type').\
        reset_index().rename_axis(None, axis=1)
        hs_1 = hscpa.loc[hscpa.jcid==city_cpa_id]
        units_added = int(hs_1.hs_change.sum())
        
        type_pivot.set_index('increment',inplace=True)
        type_pivot.fillna(0,inplace=True)
        type_pivot = type_pivot[['sch', 'jur', 'adu', 'sgoa']]
        sum_sgoa = int(type_pivot.sgoa.sum())
        sum_adu = int(type_pivot.adu.sum())
        sum_jur = int(type_pivot.jur.sum())
        sum_sch = int(type_pivot.sch.sum())

        type_pivot.sgoa.fillna(0,inplace=True)
        type_pivot.adu.fillna(0,inplace=True)
        type_pivot.jur.fillna(0,inplace=True)
        type_pivot.sch.fillna(0,inplace=True)
        if units_added == 0:
            percent_total = str(0) + '%'
            percent_sch = str(0) + '%'
            percent_jur = str(0) + '%'
            percent_adu = str(0) + '%'
            percent_sgoa = str(0) + '%'
        else:
            percent_total = str(round(units_added/units_added * 100)) + '%'
            percent_sch = str(round(sum_sch/units_added * 100)) + '%'
            percent_jur = str(round(sum_jur/units_added * 100)) + '%'
            percent_adu = str(round(sum_adu/units_added * 100)) + '%'
            percent_sgoa = str(round(sum_sgoa/units_added * 100)) + '%'
        
        assigned_df_1 = assignedcpa.loc[assignedcpa.jcid==city_cpa_id]
        assigned_capacity_1 = int(assigned_df_1.du.sum())
        assigned_capacity_adu = int(assigned_df_1.loc[assigned_df.type=='adu'].du.sum())
        assigned_capacity_sgoa = int(assigned_df_1.loc[assigned_df.type=='sgoa'].du.sum())
        sched_dev_df_1 = sunits.loc[sunits.jcid==city_cpa_id]
        sched_dev_capacity_1 = int(sched_dev_df_1.capacity_3.sum())
        capacity_df_1 = units.loc[units.jcid==city_cpa_id]
        urbansim_parcel_capacity_1 = int(capacity_df_1.loc[capacity_df_1.site_id.isnull()].capacity_2.sum())
        total_capacity_1 = assigned_capacity_1 + sched_dev_capacity_1 + urbansim_parcel_capacity_1
        remaining_total = total_capacity_1 - units_added
        remaining_sch = sched_dev_capacity_1 - sum_sch
        remaining_jur = urbansim_parcel_capacity_1 - sum_jur
        remaining_adu = assigned_capacity_adu - sum_adu
        remaining_sgoa = assigned_capacity_sgoa - sum_sgoa
        share_count = pd.DataFrame({'Total Units Added' : [sum_sch,sum_jur,sum_adu,sum_sgoa,units_added],\
                            'Percentage Units Added' : [percent_sch,percent_jur,percent_adu,\
                                                        percent_sgoa,percent_total],\
                            'Total Capacity' : [sched_dev_capacity_1,urbansim_parcel_capacity_1,\
                                               assigned_capacity_adu,assigned_capacity_sgoa,total_capacity_1],
                           'Remaining Capacity' : [remaining_sch,remaining_jur,remaining_adu,\
                                                   remaining_sgoa,remaining_total]},
                     index=['Scheduled Development','Jurisdiction Provided','Additional Dwelling Units',\
                            'SGOA Additional Capacity','Total'])
        cols = ['Total Units Added','Percentage Units Added','Total Capacity','Remaining Capacity']
        share_count = share_count[cols] 
        share_count.index.name = 'Housing Units by Capacity Type'
        fig, ax = plt.subplots(figsize=(12, 4)) # set size frame
        ax.xaxis.set_visible(False)  # hide the x axis
        ax.yaxis.set_visible(False)  # hide the y axis
        ax.set_frame_on(False)  # no visible frame, uncomment if size is ok
        tabla = table(ax, share_count, loc='upper right', colWidths=[0.18, 0.25, 0.16, 0.20])  
        # colWidths=[0.22]*len(share_count.columns)
        tabla.auto_set_font_size(False) # Activate set fontsize manually
        tabla.set_fontsize(12) # if ++fontsize is necessary ++colWidths
        #tabla.scale(1.2, 1.2) # change size table
        tabla.scale(1, 4)
        plotname = dirname + '//' + str(run_id) + '_city_cpa_' + str(city_cpa_id) + '_' + jur_ + 'percent_capacity_type_table.png'

        plt.savefig(plotname, bbox_inches="tight",transparent=True)
        plt.close(fig)

In [ ]:
dirname = os.path.join(os.getcwd(),'plots/tables3')#,datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S'))
# 'run_'+str(run_id) + '_city_cpa')
try:
    os.stat(dirname)
except:
    os.mkdir(dirname)  

In [ ]:
lambda total_chg_sr13: if total_chg_sr13 == 0: 0 else: total_chg_sr14 / total_chg_sr13 * 100.0

In [ ]:
for j, jur in enumerate(geolist):
    jur_ = jur.replace(":", "_") # colon not allowed in windows filename
    c = units_added_by_capacity_type_and_yr.loc[units_added_by_capacity_type_and_yr.jcname==jur]
    city_cpa_id = c.iloc[0]['jcid']
    # sr14_geo_df_pivot[[jur]]
    # sr13_geo_df_pivot[[jur]]
    hs_base_sr13 = int(sr13_geo_df_pivot[[jur]].loc[2012].values[0])
    hs_base_sr14 = int(sr14_geo_df_pivot[[jur]].loc[2017].values[0])
    hs_final_sr13 = int(sr13_geo_df_pivot[[jur]].loc[2050].values[0])
    hs_final_sr14 = int(sr14_geo_df_pivot[[jur]].loc[2050].values[0])
    scheddev_sr13 = int(sr13_sched_dev_totals.loc[ sr13_sched_dev_totals['geo']==jur].iloc[0]['sched_dev_sum'])
    scheddev_sr14 = int(sr14_source1.loc[sr14_source1['geo']==jur].iloc[0]['hs_sum'])
    total_chg_sr13 = int(sr13_totalchange.loc[jur][0])
    total_chg_sr14 = int(sr14_increment.loc[(sr14_increment.increment==2050) & (sr14_increment.geo==jur)].hs_cumulative)
    NOT_scheddev_sr13 = total_chg_sr13 - scheddev_sr13
    NOT_scheddev_sr14 = total_chg_sr14 - scheddev_sr14

    if hs_base_sr13 == 0:
        percent_sr13 = str(0) + '%'       
    else:
        percent_sr13 = str(round(total_chg_sr13/hs_base_sr13 * 100)) + '%'
    if hs_base_sr14 == 0:
        percent_sr14 = str(0) + '%'         
    else:
        percent_sr14 = str(round(total_chg_sr14/hs_base_sr14 * 100)) + '%'
    
    sr13_sr14_hs = pd.DataFrame({'sr13' : [hs_base_sr13,hs_final_sr13,total_chg_sr13,
                                       scheddev_sr13,NOT_scheddev_sr13,percent_sr13],\
                            'sr14' : [hs_base_sr14,hs_final_sr14,total_chg_sr14,\
                                      scheddev_sr14, NOT_scheddev_sr14,percent_sr14],
                                'chg': [hs_base_sr14 - hs_base_sr13, hs_final_sr14 - hs_final_sr13,
                                                     total_chg_sr14 - total_chg_sr13,
                                                    scheddev_sr14 - scheddev_sr13,  NOT_scheddev_sr14 - NOT_scheddev_sr13,
                                                  0]},
                     index=['Base Year Housing Units (2012/2017)','Total Housing Units (2050)',
                            'Total Change','Sched Dev',
                            'Not Sched Dev','Total Percent Change'
                            ])
    #print(sr13_sr14_hs)
    # sr13_sr14_hs
    fig, ax = plt.subplots(figsize=(8, 4)) # set size frame
    ax.xaxis.set_visible(False)  # hide the x axis
    ax.yaxis.set_visible(False)  # hide the y axis
    ax.set_frame_on(False)  # no visible frame, uncomment if size is ok
    tabla = table(ax, sr13_sr14_hs, loc='upper right', colWidths=[0.16, 0.16, 0.16])
    # colWidths=[0.22]*len(share_count.columns)
    tabla.auto_set_font_size(False) # Activate set fontsize manually
    tabla.set_fontsize(12) # if ++fontsize is necessary ++colWidths
    #tabla.scale(1.2, 1.2) # change size table
    tabla.scale(1, 4)
    plotname = dirname + '//' + str(run_id) + '_city_cpa_' + str(city_cpa_id) + '_' + jur_ + 'forecast_comparison.png'
    #plotname = dirname + '//' + str(run_id) + '_city_cpa_' + str(city_cpa_id) + '_' + jur_ + 'percent_capacity_type_table.png'
    plt.savefig(plotname, bbox_inches="tight",transparent=True)
    plt.close(fig)

In [ ]:
# capacity_df_1 = units.loc[units.cap_jurisdiction_id==city_cpa_id]

In [ ]:
# capacity_df_1

In [ ]:
# city_cpa_id

In [ ]:
# hs.head()

In [ ]:
city_cpa_id = 1467
sched_dev_df_1 = sched_dev_df.loc[sched_dev_df.cap_jurisdiction_id==city_cpa_id]
sched_dev_capacity_1 = int(sched_dev_df_1.capacity_3.sum())

In [ ]:
sched_dev_capacity_1

In [ ]:
jur='NCFUA Reserve'
c = units_added_by_capacity_type_and_yr.loc[units_added_by_capacity_type_and_yr.jcname==jur]
type_pivot = c.pivot(index='increment', columns='capacity_type', values='units_by_type').\
reset_index().rename_axis(None, axis=1)

In [ ]:
capacity_df_1 = capacity_df.loc[capacity_df.cap_jurisdiction_id==1467]

In [ ]:
capacity_df_1

In [ ]:
sum_sch = int(type_pivot.sch.sum())
sum_sch

In [ ]:
# type_pivot

In [ ]:
assigned_df.head()

In [ ]:
int(city_cpa_id)

In [ ]:
assigned_df_1 = assigned_df.loc[assigned_df.cap_jurisdiction_id==int(city_cpa_id)]

In [ ]:
type_pivot.head()

## bar chart

In [ ]:
hs2 = hs.copy()
hs2.replace('cc', 'sgoa',inplace=True)
hs2.replace('mc', 'sgoa',inplace=True)
hs2.replace('tc', 'sgoa',inplace=True)
hs2.replace('tco', 'sgoa',inplace=True)
hs2.replace('uc', 'sgoa',inplace=True)

In [ ]:
bins = range(2015,2055,5)
names = [str(x) for x in range(2020,2055,5)]

In [ ]:
hs2['increment'] = pd.cut(hs2.year_simulation, bins, labels=names)
# hs2.head()

In [ ]:
units_added_by_capacity_type_and_yr = pd.DataFrame({'units_by_type': hs2.
                                          groupby(["increment","capacity_type"])
                                          .hs_change.sum()}).reset_index()
# units_added_by_capacity_type_and_yr.head()

In [ ]:
units_added_by_capacity_type_and_yr.head()

In [ ]:
type_pivot = units_added_by_capacity_type_and_yr.pivot\
(index='increment', columns='capacity_type', values='units_by_type').\
reset_index().rename_axis(None, axis=1)
# type_pivot.head()

In [ ]:
sum_sgoa = int(type_pivot.sgoa.sum())
sum_adu = int(type_pivot.adu.sum())
sum_jur = int(type_pivot.jur.sum())
sum_sch = int(type_pivot.sch.sum())

type_pivot.sgoa.fillna(0,inplace=True)
type_pivot.adu.fillna(0,inplace=True)
type_pivot.jur.fillna(0,inplace=True)
type_pivot.sch.fillna(0,inplace=True)

In [ ]:
type_pivot.set_index('increment',inplace=True)
type_pivot = type_pivot[['sch', 'jur', 'adu', 'sgoa']]
type_pivot['sch'] = type_pivot['sch'].cumsum()
type_pivot['jur'] = type_pivot['jur'].cumsum()
type_pivot['adu'] = type_pivot['adu'].cumsum()
type_pivot['sgoa'] = type_pivot['sgoa'].cumsum()
# type_pivot.head()

In [ ]:
parcel_capacity_sql = '''
    SELECT [parcel_id],[jurisdiction_id],[cap_jurisdiction_id],[site_id],
           [du_2015],[du_2017],[capacity_1],[capacity_2]
      FROM [urbansim].[urbansim].[parcel]'''
capacity_df = pd.read_sql(parcel_capacity_sql,mssql_engine)
capacity_df_1 = capacity_df.loc[capacity_df.cap_jurisdiction_id==1]
urbansim_parcel_capacity_1 = int(capacity_df_1.loc[capacity_df_1.site_id.isnull()].capacity_2.sum())
print("\nCapacity from urbansim.parcel where site id is null: {:,}".format(urbansim_parcel_capacity_1))
# 291,989

In [ ]:
sched_dev_parcel_sql = '''
SELECT s.parcel_id, p.jurisdiction_id, p.cap_jurisdiction_id,capacity_3 
FROM [urbansim].[urbansim].[scheduled_development_parcel] s
JOIN [urbansim].[urbansim].[parcel] p
ON p.parcel_id = s.parcel_id'''
sched_dev_df = pd.read_sql(sched_dev_parcel_sql, mssql_engine)
sched_dev_df_1 = sched_dev_df.loc[sched_dev_df.cap_jurisdiction_id==1]
sched_dev_capacity_1 = int(sched_dev_df_1.capacity_3.sum())
print("\nScheduled development capacity from urbansim.scheduled_development_parcel : {:,}".\
      format(sched_dev_capacity_1))

In [ ]:
assigned_parcel_sql = '''
SELECT  a.parcel_id, cap_jurisdiction_id, jurisdiction_id, a.du, a.type
   FROM [urbansim].[urbansim].[additional_capacity] a
   JOIN urbansim.parcel p on p.parcel_id = a.parcel_id
  WHERE version_id = %s'''
assigned_parcel_sql = assigned_parcel_sql % versions['additional_capacity_version']
assigned_df = pd.read_sql(assigned_parcel_sql, mssql_engine)
assigned_df.replace('cc', 'sgoa',inplace=True)
assigned_df.replace('mc', 'sgoa',inplace=True)
assigned_df.replace('tc', 'sgoa',inplace=True)
assigned_df.replace('tco', 'sgoa',inplace=True)
assigned_df.replace('uc', 'sgoa',inplace=True)
assigned_df_1 = assigned_df.loc[assigned_df.cap_jurisdiction_id==1]
assigned_capacity_1 = int(assigned_df_1.du.sum())
assigned_capacity_adu = int(assigned_df_1.loc[assigned_df.type=='adu'].du.sum())
assigned_capacity_sgoa = int(assigned_df_1.loc[assigned_df.type=='sgoa'].du.sum())

print("\nCapacity from urbansim.additional_capacity (ADU and SGOAs): {:,}".format(assigned_capacity_1))
print("\nCapacity from urbansim.additional_capacity (ADU): {:,}".format(assigned_capacity_adu))
print("\nCapacity from urbansim.additional_capacity (SGOAs): {:,}".format(assigned_capacity_sgoa))
# assigned_df.head()

In [ ]:
total_capacity_1 = assigned_capacity_1 + sched_dev_capacity_1 + urbansim_parcel_capacity_1
total_capacity_1 

In [ ]:
remaining_total = total_capacity_1 - units_added
remaining_sch = sched_dev_capacity_1 - sum_sch
remaining_jur = urbansim_parcel_capacity_1 - sum_jur
remaining_adu = assigned_capacity_adu - sum_adu
remaining_sgoa = assigned_capacity_sgoa - sum_sgoa
percent_total = str(round(units_added/units_added * 100)) + '%'
percent_sch = str(round(sum_sch/units_added * 100)) + '%'
percent_jur = str(round(sum_jur/units_added * 100)) + '%'
percent_adu = str(round(sum_adu/units_added * 100)) + '%'
percent_sgoa = str(round(sum_sgoa/units_added * 100)) + '%'

In [ ]:
share_count = pd.DataFrame({'Total Units Added' : [sum_sch,sum_jur,sum_adu,sum_sgoa,units_added],\
                            'Percentage Units Added' : [percent_sch,percent_jur,percent_adu,\
                                                        percent_sgoa,percent_total],\
                            'Total Capacity' : [sched_dev_capacity_1,urbansim_parcel_capacity_1,\
                                               assigned_capacity_adu,assigned_capacity_sgoa,total_capacity_1],
                           'Remaining Capacity' : [remaining_sch,remaining_jur,remaining_adu,\
                                                   remaining_sgoa,remaining_total]},
                     index=['Scheduled Development','Jurisdiction Provided','Additional Dwelling Units',\
                            'SGOA Additional Capacity','Total'])
cols = ['Total Units Added','Percentage Units Added','Total Capacity','Remaining Capacity']
share_count = share_count[cols] 

In [ ]:
share_count.index.name = 'Housing Units by Capacity Type'
share_count

In [ ]:

fig, ax = plt.subplots(figsize=(12, 4)) # set size frame
ax.xaxis.set_visible(False)  # hide the x axis
ax.yaxis.set_visible(False)  # hide the y axis
ax.set_frame_on(False)  # no visible frame, uncomment if size is ok
tabla = table(ax, share_count, loc='upper right', colWidths=[0.18, 0.25, 0.16, 0.20])  
# colWidths=[0.22]*len(share_count.columns)
tabla.auto_set_font_size(False) # Activate set fontsize manually
tabla.set_fontsize(12) # if ++fontsize is necessary ++colWidths
#tabla.scale(1.2, 1.2) # change size table
tabla.scale(1, 4)
plotname = dirname + '//' + str(run_id) + '_city_cpa_' + city_cpa_id + '_' + jur_ + '_percent_capacity_type_table.png'

plt.savefig(plotname, bbox_inches="tight",transparent=True)

In [ ]:
share_count.columns

In [ ]:
jur

In [ ]:
sr14_geo_df_pivot[[jur]]

In [ ]:
sr14_geo_df_pivot[[jur]]

In [ ]:
# sr14_geo_df_pivot[[jur]]
# sr13_geo_df_pivot[[jur]]
hs_base_sr13 = int(sr13_geo_df_pivot[[jur]].loc[2012].values[0])
hs_base_sr14 = int(sr14_geo_df_pivot[[jur]].loc[2017].values[0])
hs_final_sr13 = int(sr13_geo_df_pivot[[jur]].loc[2050].values[0])
hs_final_sr14 = int(sr14_geo_df_pivot[[jur]].loc[2050].values[0])
scheddev_sr13 = int(sr13_sched_dev_totals.loc[ sr13_sched_dev_totals['geo']==jur].iloc[0]['sched_dev_sum'])
scheddev_sr14 = int(sr14_source1.loc[sr14_source1['geo']==jur].iloc[0]['hs_sum'])
total_chg_sr13 = int(sr13_totalchange.loc[jur][0])
total_chg_sr14 = int(sr14_increment.loc[(sr14_increment.increment==2050) & (sr14_increment.geo==jur)].hs_cumulative)
NOT_scheddev_sr13 = total_chg_sr13 - scheddev_sr13
NOT_scheddev_sr14 = total_chg_sr14 - scheddev_sr14
percent_sr13 = str(round(total_chg_sr13/hs_base_sr13 * 100)) + '%'
percent_sr14 = str(round(total_chg_sr14/hs_base_sr14 * 100)) + '%'
sr13_sr14_hs = pd.DataFrame({'sr13' : [hs_base_sr13,hs_final_sr13,total_chg_sr13,
                                       scheddev_sr13,NOT_scheddev_sr13,percent_sr13],\
                            'sr14' : [hs_base_sr14,hs_final_sr14,total_chg_sr14,\
                                      scheddev_sr14, NOT_scheddev_sr14,percent_sr14]},
                     index=['Total Housing Units (2012/2017)','Total Housing Units (2050)',
                            'Total Change','Sched Dev',
                            'Not Sched Dev','Total Percent Change'
                            ])
sr13_sr14_hs

In [ ]:
fig, ax = plt.subplots(figsize=(8, 4)) # set size frame
ax.xaxis.set_visible(False)  # hide the x axis
ax.yaxis.set_visible(False)  # hide the y axis
ax.set_frame_on(False)  # no visible frame, uncomment if size is ok
tabla = table(ax, sr13_sr14_hs, loc='upper right', colWidths=[0.16, 0.16])  
# colWidths=[0.22]*len(share_count.columns)
tabla.auto_set_font_size(False) # Activate set fontsize manually
tabla.set_fontsize(12) # if ++fontsize is necessary ++colWidths
#tabla.scale(1.2, 1.2) # change size table
tabla.scale(1, 4)
plt.savefig('plots/test/forecast_comparison.png', bbox_inches="tight",transparent=True)

In [ ]:
# sr14_geo_df_pivot[[jur]]
# sr13_geo_df_pivot[[jur]]
hs_base_sr13 = int(sr13_geo_df_pivot[[jur]].loc[2012].values[0])
hs_base_sr14 = int(sr14_geo_df_pivot[[jur]].loc[2017].values[0])
hs_final_sr13 = int(sr13_geo_df_pivot[[jur]].loc[2050].values[0])
hs_final_sr14 = int(sr14_geo_df_pivot[[jur]].loc[2050].values[0])
scheddev_sr13 = int(sr13_sched_dev_totals.loc[ sr13_sched_dev_totals['geo']==jur].iloc[0]['sched_dev_sum'])
scheddev_sr14 = int(sr14_source1.loc[sr14_source1['geo']==jur].iloc[0]['hs_sum'])
total_chg_sr13 = int(sr13_totalchange.loc[jur][0])
total_chg_sr14 = int(sr14_increment.loc[(sr14_increment.increment==2050) & (sr14_increment.geo==jur)].hs_cumulative)
NOT_scheddev_sr13 = total_chg_sr13 - scheddev_sr13
NOT_scheddev_sr14 = total_chg_sr14 - scheddev_sr14
percent_sr13 = str(round(total_chg_sr13/hs_base_sr13 * 100)) + '%'
percent_sr14 = str(round(total_chg_sr14/hs_base_sr14 * 100)) + '%'
sr13_sr14_hs = pd.DataFrame({'sr13' : [hs_base_sr13,hs_final_sr13,total_chg_sr13,
                                       scheddev_sr13,NOT_scheddev_sr13,percent_sr13],\
                            'sr14' : [hs_base_sr14,hs_final_sr14,total_chg_sr14,\
                                      scheddev_sr14, NOT_scheddev_sr14,percent_sr14]},
                     index=['Total Housing Units (2012/2017)','Total Housing Units (2050)',
                            'Total Change','Sched Dev',
                            'Not Sched Dev','Total Percent Change'
                            ])
sr13_sr14_hs
fig, ax = plt.subplots(figsize=(8, 4)) # set size frame
ax.xaxis.set_visible(False)  # hide the x axis
ax.yaxis.set_visible(False)  # hide the y axis
ax.set_frame_on(False)  # no visible frame, uncomment if size is ok
tabla = table(ax, sr13_sr14_hs, loc='upper right', colWidths=[0.16, 0.16])  
# colWidths=[0.22]*len(share_count.columns)
tabla.auto_set_font_size(False) # Activate set fontsize manually
tabla.set_fontsize(12) # if ++fontsize is necessary ++colWidths
#tabla.scale(1.2, 1.2) # change size table
tabla.scale(1, 4)
plt.savefig('plots/test/forecast_comparison.png', bbox_inches="tight",transparent=True)

#### old

In [ ]:
units_added_by_capacity_type_and_yr = pd.DataFrame({'units_by_type': hs2.
                                          groupby(["year_simulation","capacity_type"])
                                          .hs_change.sum()}).reset_index()
type_pivot2 = units_added_by_capacity_type_and_yr.pivot\
(index='year_simulation', columns='capacity_type', values='units_by_type').\
reset_index().rename_axis(None, axis=1)
type_pivot2.fillna(0,inplace=True)
type_pivot2.set_index('year_simulation',inplace=True)
type_pivot2 = type_pivot2[['sch', 'jur', 'adu', 'sgoa']]

In [ ]:

type_pivot2['sch'] = type_pivot2['sch'].cumsum()
type_pivot2['jur'] = type_pivot2['jur'].cumsum()
type_pivot2['adu'] = type_pivot2['adu'].cumsum()
type_pivot2['sgoa'] = type_pivot2['sgoa'].cumsum()

city_cpa_id = 1
jur = 'Carlsbad'



ax = type_pivot2.plot.bar(stacked=True)
ptitle = 'DRAFT Cumulative Housing Units Added By Capacity Type\n' + jur + ' (' +  str(city_cpa_id) + ')'
ax.set_ylabel("housing units added",size=14)
ax.set_title(ptitle,size=16) 
orig = ax.legend
# ax.legend(["sch (5)", "jur (10)","adu (20)","SGOA (15)"]);

In [ ]:


city_cpa_id = 1
jur = 'Carlsbad'

ax = type_pivot2.plot.bar(stacked=True)
handles, labels = ax.get_legend_handles_labels()
extra = Rectangle((0, 0), 1, 1, fc="w", fill=False, edgecolor='none', linewidth=0)
handles.insert(0,extra)
labels.insert(0,'(added/capacity)')
labels = ['Total (added/capacity)\n (' + str(units_added) + '/' + str(total_capacity_1) +  ')',\
          'sch (' + str(sum_sch) + '/' + str(sched_dev_capacity_1) + ')',\
          'jur (' + str(sum_jur) + '/' + str(urbansim_parcel_capacity_1) + ')',\
          'adu (' + str(sum_adu) + '/' + str(assigned_capacity_adu) + ')',\
          'SGOA (' + str(sum_sgoa) + '/' + str(assigned_capacity_sgoa) + ')']
ptitle = 'DRAFT Cumulative Housing Units Added By Capacity Type\n' + jur + ' (citycpa_id=' +  str(city_cpa_id) + ')'
ax.set_ylabel("housing units added",size=14)
ax.set_title(ptitle,size=16)
leg = ax.legend(handles, labels,fontsize=12)
# ax.set_xlabel("increment")

ax.set_xlabel("increment\n\n increment 2020 = (2017,2018,2019,2020)             \nincrement 2025 = (2021, 2022, 2023, 2024, 2025)\
\nincrement 2030 = (2026, 2027, 2028, 2029, 2030)\nincrement 2035 = (2031, 2032, 2033, 2034, 2035)\
\nincrement 2040 = (2036, 2037, 2038, 2039, 2040)\nincrement 2045 = (2041, 2042, 2043, 2044, 2045)\
\nincrement 2050 = (2046, 2047, 2048, 2049, 2050)")
# ax.legend(["sch (5)", "jur (10)","adu (20)","SGOA (15)"]);


In [ ]:
capty = pd.DataFrame({'Total' : [units_added,remaining_total,total_capacity_1],'sch' : [sum_sch, 0, sched_dev_capacity_1],\
                      'jur' : [sum_jur, 0, urbansim_parcel_capacity_1],\
                      'adu' : [sum_adu,618, assigned_capacity_adu],'sgoa' : [sum_sgoa, 160, assigned_capacity_sgoa]},\
                     index=['Units Added','Remaining Capacity','Total Capacity'])

cols = ['Total','sch','jur','adu','sgoa']
capty = capty[cols] 